In [52]:
import pandas as pd
df= pd.read_csv(r"C:\Users\Access\Documents\AI ROBOT\XAUUSD_5m_trade_result_shooting_trade_list.csv" )

In [53]:
df

,Unnamed: 0,Gmt time,Open,High,Low,Close,trade_type_shooting,trade_result_shooting,tp_shooting,sl_shooting
0,0,2020-01-01 00:00:00,1516.915,1516.915,1516.915,1516.915,0,0,NaN,NaN
1,1,2020-01-01 00:05:00,1516.915,1516.915,1516.915,1516.915,0,0,NaN,NaN
2,2,2020-01-01 00:10:00,1516.915,1516.915,1516.915,1516.915,0,0,NaN,NaN
3,3,2020-01-01 00:15:00,1516.915,1516.915,1516.915,1516.915,0,0,NaN,NaN
4,4,2020-01-01 00:20:00,1516.915,1516.915,1516.915,1516.915,0,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
526171,526171,2024-12-31 23:35:00,2624.255,2624.255,2624.255,2624.255,0,0,NaN,NaN
526172,526172,2024-12-31 23:40:00,2624.255,2624.255,2624.255,2624.255,0,0,NaN,NaN
526173,526173,2024-12-31 23:45:00,2624.255,2624.255,2624.255,2624.255,0,0,NaN,NaN
526174,526174,2024-12-31 23:50:00,2624.255,2624.255,2624.255,2624.255,0,0,NaN,NaN


In [55]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam

df = df[df['trade_result_shooting'] != 0]

X = df[['Open','High','Low','Close','trade_type_shooting']]

y = df['trade_result_shooting']


# تصفية البيانات لإزالة الصفقات التي لا يوجد فيها تداول (0)
mask = (y == 1) | (y == 2)
X_filtered = X[mask]
y_filtered = y[mask]

# تحويل التسميات إلى 0 و1 (1 => 0 خاسرة، 2 => 1 رابحة)
y_filtered = np.where(y_filtered == 1, 0, 1)

# تسوية البيانات
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X_filtered)

# دالة لإنشاء تسلسلات زمنية
def create_sequences(data, targets, window_size):
    X_seq, y_seq = [], []
    for i in range(len(data) - window_size):
        X_seq.append(data[i:i+window_size])
        y_seq.append(targets[i+window_size])
    return np.array(X_seq), np.array(y_seq)

# إنشاء تسلسلات بحجم نافذة 3
window_size = 3
X_seq, y_seq = create_sequences(X_scaled, y_filtered, window_size)

# تقسيم البيانات مع الحفاظ على الترتيب الزمني
train_size = int(0.7 * len(X_seq))
X_train, X_test = X_seq[:train_size], X_seq[train_size:]
y_train, y_test = y_seq[:train_size], y_seq[train_size:]

# بناء نموذج LSTM
model = Sequential()
model.add(LSTM(128, 
               input_shape=(window_size, X_train.shape[2]), 
               return_sequences=True))
model.add(Dropout(0.1))
model.add(LSTM(32))
#model.add(Dropout(0.1))
model.add(Dense(1, activation='sigmoid'))  # طبقة إخراج واحدة للتصنيف الثنائي

# تجميع النموذج
model.compile(loss='binary_crossentropy',
              optimizer=Adam(learning_rate=0.01),
              metrics=['accuracy'])

# إضافة إيقاف مبكر
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# تدريب النموذج مع موازنة الفئات
history = model.fit(X_train, y_train,
                    epochs=20,
                    batch_size=100,
                    validation_split=0.2,
                    callbacks=[early_stop],
                    class_weight={0: 5, 1: 1},  # زيادة وزن الصفقات الرابحة
                    verbose=1)

# تقييم النموذج
y_pred = (model.predict(X_test) > 0.9).astype(int)
print("\nتقرير التصنيف:")
print(classification_report(y_test, y_pred, target_names=['loss', 'win']))

C:\Users\Access\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 18s 110ms/step - accuracy: 0.3891 - loss: 1.2310 - val_accuracy: 0.1902 - val_loss: 0.7789
Epoch 2/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - accuracy: 0.1872 - loss: 1.2110 - val_accuracy: 0.1902 - val_loss: 0.7739
Epoch 3/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - accuracy: 0.2012 - loss: 1.2397 - val_accuracy: 0.1902 - val_loss: 0.7247
Epoch 4/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - accuracy: 0.1855 - loss: 1.2063 - val_accuracy: 0.1902 - val_loss: 0.7367
Epoch 5/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - accuracy: 0.2011 - loss: 1.2414 - val_accuracy: 0.1902 - val_loss: 0.7436
Epoch 6/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - accuracy: 0.1925 - loss: 1.2200 - val_accuracy: 0.1902 - val_loss: 0.7605
Epoch 7/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - accuracy: 0.1931 - loss: 1.2214 - val_accuracy: 0.1902 - val_loss: 0.7620
Epoch 8/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - accuracy: 0.1941 - loss: 1.2234 - val_accuracy: 0.1902 -

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [57]:
# حفظ النموذج كاملاً
model.save(r"C:\Users\Access\Documents\AI ROBOT\my_lstm_model_shooting.keras")

In [ ]:
from keras.models import load_model

# تحميل النموذج
loaded_model = load_model(r"C:\Users\Access\Documents\AI ROBOT\my_lstm_model_shooting.keras")

# استخدام النموذج المحمل
y_pred_loaded = (loaded_model.predict(X_test) > 0.9).astype(int)